In [1]:
import sys
from pyspark.sql import S
from pyspark.sql.functions import *
from datetime import date, timedelta

In [2]:
spark = SparkSession \
    .builder \
    .appName("klaus_session") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/19 03:33:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/19 03:33:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [15]:
df_silver = spark.read.json("/opt/airflow/bronze",multiLine=True)

In [16]:
record_count = df_silver.count()

if record_count > 0:
    print("Dataframe has data, processing...")
else:
    message = "Dataframe is empty, notifying data engineering team..."
    notification_discord(message)
    sys.exit()

Dataframe has data, processing...


In [17]:
df_silver.show(vertical=True)

-RECORD 0------------------------------
 address_1      | 1716 Topeka St       
 address_2      | NULL                 
 address_3      | NULL                 
 brewery_type   | micro                
 city           | Norman               
 country        | United States        
 id             | 5128df48-79fc-4f0... 
 latitude       | 35.25738891          
 longitude      | -97.46818222         
 name           | (405) Brewing Co     
 phone          | 4058160490           
 postal_code    | 73069-8224           
 state          | Oklahoma             
 state_province | Oklahoma             
 street         | 1716 Topeka St       
 website_url    | http://www.405bre... 
-RECORD 1------------------------------
 address_1      | 407 Radam Ln Ste ... 
 address_2      | NULL                 
 address_3      | NULL                 
 brewery_type   | micro                
 city           | Austin               
 country        | United States        
 id             | 9c5a66c8-cc13-416... 


In [18]:
df_silver = df_silver.select(col('id'),
                             col('name'),
                             col('brewery_type'),  
                             col('phone'),
                             col('country'),
                             col('state'),
                             col('state_province'),
                             col('city'),
                             col('postal_code'),
                             col('latitude'),
                             col('longitude'),
                             col('street'),
                             col('address_1'),
                             col('address_2'),
                             col('address_3')
                             )

In [10]:
df_silver.show(1)

+--------------------+----------------+------------+-------------+---------------+-------------+----------------------+------------+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+
|          id_brewery|    name_brewery|type_brewery|phone_brewery|country_brewery|state_brewery|state_province_brewery|city_brewery|postal_code_brewery|latitude_brewery|longitude_brewery|address_1_brewery|address_2_brewery|address_3_brewery|
+--------------------+----------------+------------+-------------+---------------+-------------+----------------------+------------+-------------------+----------------+-----------------+-----------------+-----------------+-----------------+
|5128df48-79fc-4f0...|(405) Brewing Co|       micro|   4058160490|  United States|     Oklahoma|              Oklahoma|      Norman|         73069-8224|     35.25738891|     -97.46818222|   1716 Topeka St|             NULL|             NULL|
+--------------------+----------

In [19]:
df_silver = df_silver \
    .withColumnRenamed("id", "id_brewery") \
    .withColumnRenamed("name", "name_brewery") \
    .withColumnRenamed("brewery_type", "type_brewery") \
    .withColumnRenamed("address_1", "address_1_brewery") \
    .withColumnRenamed("address_2", "address_2_brewery") \
    .withColumnRenamed("address_3", "address_3_brewery") \
    .withColumnRenamed("street", "street_brewery") \
    .withColumnRenamed("city", "city_brewery") \
    .withColumnRenamed("country", "country_brewery") \
    .withColumnRenamed("state", "state_brewery") \
    .withColumnRenamed("state_province", "state_province_brewery") \
    .withColumnRenamed("phone", "phone_brewery") \
    .withColumnRenamed("latitude", "latitude_brewery") \
    .withColumnRenamed("longitude", "longitude_brewery") \
    .withColumnRenamed("postal_code", "postal_code_brewery")

In [20]:
columns_to_check_null = [
    'id_brewery',
    'name_brewery',
    'street_brewery',
    'type_brewery',
    'phone_brewery',
    'country_brewery',
    'state_brewery',
    'state_province_brewery',
    'city_brewery',
    'postal_code_brewery',
    'latitude_brewery',
    'longitude_brewery',
    'address_1_brewery',
    'address_2_brewery',
    'address_3_brewery'
]

In [23]:
null_counts = []

for column in columns_to_check_null:
    null_count_expr = sum(when(col(column).isNull(), 1).otherwise(0)).alias(column)
    null_counts.append(null_count_expr)

null_counts_df = df_silver.select(null_counts)

In [24]:
null_counts_df.show(vertical=True)

-RECORD 0---------------------
 id_brewery             | 0   
 name_brewery           | 0   
 street_brewery         | 3   
 type_brewery           | 0   
 phone_brewery          | 5   
 country_brewery        | 0   
 state_brewery          | 0   
 state_province_brewery | 0   
 city_brewery           | 0   
 postal_code_brewery    | 0   
 latitude_brewery       | 10  
 longitude_brewery      | 10  
 address_1_brewery      | 3   
 address_2_brewery      | 49  
 address_3_brewery      | 50  



In [25]:
df_silver.filter(col('phone_brewery').isNull()).show()

+--------------------+--------------------+------------+-------------+---------------+-------------+----------------------+------------+-------------------+----------------+-----------------+--------------------+--------------------+-----------------+-----------------+
|          id_brewery|        name_brewery|type_brewery|phone_brewery|country_brewery|state_brewery|state_province_brewery|city_brewery|postal_code_brewery|latitude_brewery|longitude_brewery|      street_brewery|   address_1_brewery|address_2_brewery|address_3_brewery|
+--------------------+--------------------+------------+-------------+---------------+-------------+----------------------+------------+-------------------+----------------+-----------------+--------------------+--------------------+-----------------+-----------------+
|0faa0fb2-fffa-416...|12 West Brewing C...|       micro|         NULL|  United States|      Arizona|               Arizona|        Mesa|              85207|       33.436188|     -111.5860662

In [26]:
df_silver.filter(col('phone_brewery').isNotNull()).show()

+--------------------+--------------------+------------+-------------+---------------+-------------+----------------------+--------------+-------------------+-----------------+------------------+--------------------+--------------------+-----------------+-----------------+
|          id_brewery|        name_brewery|type_brewery|phone_brewery|country_brewery|state_brewery|state_province_brewery|  city_brewery|postal_code_brewery| latitude_brewery| longitude_brewery|      street_brewery|   address_1_brewery|address_2_brewery|address_3_brewery|
+--------------------+--------------------+------------+-------------+---------------+-------------+----------------------+--------------+-------------------+-----------------+------------------+--------------------+--------------------+-----------------+-----------------+
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|   4058160490|  United States|     Oklahoma|              Oklahoma|        Norman|         73069-8224|      35.25738891|  

In [27]:
df_silver = df_silver.withColumn("phone_brewery", when(col("phone_brewery").isNull(), "00000000000").otherwise(col("phone_brewery")))

In [28]:
df_silver.filter(col('latitude_brewery').isNull()).show()

+--------------------+--------------------+------------+-------------+---------------+--------------+----------------------+-------------+-------------------+----------------+-----------------+--------------------+--------------------+-----------------+-----------------+
|          id_brewery|        name_brewery|type_brewery|phone_brewery|country_brewery| state_brewery|state_province_brewery| city_brewery|postal_code_brewery|latitude_brewery|longitude_brewery|      street_brewery|   address_1_brewery|address_2_brewery|address_3_brewery|
+--------------------+--------------------+------------+-------------+---------------+--------------+----------------------+-------------+-------------------+----------------+-----------------+--------------------+--------------------+-----------------+-----------------+
|9c5a66c8-cc13-416...|    (512) Brewing Co|       micro|   5129211545|  United States|         Texas|                 Texas|       Austin|         78745-1197|            NULL|         

In [29]:
df_silver.filter(col('latitude_brewery').isNotNull()).show()

+--------------------+--------------------+------------+-------------+---------------+-------------+----------------------+--------------+-------------------+-----------------+------------------+--------------------+--------------------+-----------------+-----------------+
|          id_brewery|        name_brewery|type_brewery|phone_brewery|country_brewery|state_brewery|state_province_brewery|  city_brewery|postal_code_brewery| latitude_brewery| longitude_brewery|      street_brewery|   address_1_brewery|address_2_brewery|address_3_brewery|
+--------------------+--------------------+------------+-------------+---------------+-------------+----------------------+--------------+-------------------+-----------------+------------------+--------------------+--------------------+-----------------+-----------------+
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|   4058160490|  United States|     Oklahoma|              Oklahoma|        Norman|         73069-8224|      35.25738891|  

In [30]:
df_silver.count()

50

In [31]:
# if df_silver.filter(df_silver.name_state.isNull()).count() >= 1 or df_silver.filter(df_silver.brewery_type.isNull()).count() >= 1:
#     print("has null values")
#     sys.exit()
# else:
#     print("has no null values")

In [32]:
#Add column for CDC control
df_silver = df_silver.withColumn("updated_at", date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss"))

In [33]:
df_silver.show(1,vertical=True,truncate=False)

-RECORD 0------------------------------------------------------
 id_brewery             | 5128df48-79fc-4f0f-8b52-d06be54d0cec 
 name_brewery           | (405) Brewing Co                     
 type_brewery           | micro                                
 phone_brewery          | 4058160490                           
 country_brewery        | United States                        
 state_brewery          | Oklahoma                             
 state_province_brewery | Oklahoma                             
 city_brewery           | Norman                               
 postal_code_brewery    | 73069-8224                           
 latitude_brewery       | 35.25738891                          
 longitude_brewery      | -97.46818222                         
 street_brewery         | 1716 Topeka St                       
 address_1_brewery      | 1716 Topeka St                       
 address_2_brewery      | NULL                                 
 address_3_brewery      | NULL          

In [34]:
today = date.today()

In [36]:
#Writing DataFrame
df_silver.write \
.format("parquet") \
.mode("append") \
.partitionBy("state_brewery") \
.option("mergeSchema", "true") \
.save(f'/opt/airflow/silver/')